In [1]:
import spacy
import numpy as np
import pandas as pd

from extract_load import multiple_csv_to_dict_df
from preprocessing import combine_dataframes

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error

C:\Users\jvaug_000\Anaconda3\envs\group3nlp\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('en_core_web_lg')

ModuleNotFoundError: No module named 'biomedicalWordVectors'

In [4]:
nlp.vocab.vectors_length

300

*Load data*

In [5]:
path = "./data"
dict_df = multiple_csv_to_dict_df(path)

#key_filter = ["2014","2015","2016"]
#dict_df = { key: dict_df[key] for key in key_filter}

#d1 = combine_dataframes(dict_df)
d1 = dict_df['2015']

## Create word_vectors for each doc


In [7]:

#d1['doc_vector'] = 1
data = d1.loc[:,['HCPCS Description','Average Medicare Standardized Payment Amount']]

train_data, validation_data = train_test_split(data, test_size=.10, random_state=0)

#train_text_vector = train_data['HCPCS Description'].map(lambda description: nlp(description).vector)
train_target = train_data["Average Medicare Standardized Payment Amount"]

embedding_dim = 300
n_sentences = len(train_data)

X = np.zeros((n_sentences, embedding_dim))

for idx, hcpcs_description in enumerate(train_data['HCPCS Description']): 
    
    doc = nlp(hcpcs_description)
    X[idx, :] = doc.vector
    #d1['word_vector'][idx] = 2
    
    

In [8]:
X.shape

(11427, 300)

In [9]:
train_data.shape

(11427, 2)

In [10]:
# Cross-validate Predict
model = RandomForestRegressor(n_estimators=10, max_depth=5)
predict = cross_val_predict(model, X,train_target)
#predict_processed = [int(round(i)) for i in predict]

# Analyze accuracy
#print("Accuracy: %.2f percent" % (accuracy_score(train_target, predict_processed) * 100))
# Analyze Root Mean Squared Error
print("RMSE: %.2f " % np.sqrt(mean_squared_error(train_target, predict)))

RMSE: 656.77 


RMSE: 633.21 cross val no grid sm spacy model random forest


In [17]:
doc = nlp(train_data['HCPCS Description'][0])
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

()

In [20]:
doc2 = nlp("Apple is looking at buying U.K. startup for $1 billion")
for ent in doc2.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY
